## Food.com Dataset Reccomender Binary Matrix
Below is data exploration on the foolwoing dataset https://www.kaggle.com/datasets/shuyangli94/food-com-recipes-and-user-interactions?select=RAW_recipes.csv 

This dataset consists of 180K+ recipes and 700K+ recipe reviews covering 18 years of user interactions and uploads on Food.com 

In [ ]:
import numpy as np
import zipfile
from tabulate import tabulate
import pandas as pd
import matplotlib.pyplot as plt
import os
import warnings
import nltk
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk.corpus import stopwords
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MultiLabelBinarizer

#from surprise import SVD
#from surprise import Dataset
#from surprise import Reader
#from surprise.model_selection import train_test_split
#from surprise import accuracy
#import tensorflow.keras as tf

%matplotlib inline


## Loading the Dataset
From the zipped dataset we extract the recipes and the interactions (reviews).
Below is how we got the data from the original very large dataset.

In [ ]:
# zip_file_path = "archive.zip"
# csv_file_name1 = 'RAW_recipes.csv'
# csv_file_name2 = 'RAW_interactions.csv' 

# # Initialize two DataFrames to store the data from the two CSV files
# df1 = None
# df2 = None

# # Open the zip file and read the first CSV file into the first DataFrame
# with zipfile.ZipFile(zip_file_path, 'r') as zip_file:
#     with zip_file.open(csv_file_name1) as csv_file_in_zip:
#         df_recipes = pd.read_csv(csv_file_in_zip)

# # Open the zip file again and read the second CSV file into the second DataFrame
# with zipfile.ZipFile(zip_file_path, 'r') as zip_file:
#     with zip_file.open(csv_file_name2) as csv_file_in_zip:
#         df_reviews = pd.read_csv(csv_file_in_zip)

# #We renamed the id column to match to id in reviews so we can merge on this later
# df_recipes.rename(columns={'id': 'recipe_id'}, inplace=True)




In [ ]:
# #We combine both the dataframes

# combined_df = pd.merge(df_reviews, df_recipes, on='recipe_id', how='inner')


## Data Selection

Below we search the tags that indicate the recipe is for a cocktail. This cuts downs are dataset from 180K recipes to around 4K, significiantly reducing our file size. We have used this exported csv as our base dataset so we don't have to rely on git lfs.

In [ ]:
# combined_df = combined_df[combined_df['tags'].str.contains('cocktails', case=False, na=False)]

# combined_df.rename(columns={'id': 'recipe_id'}, inplace=True)

# #We drop columns that we don't need
# columns_to_drop = ['date', 'review','minutes','nutrition','contributor_id', 'description','submitted']

# combined_df = combined_df.drop(columns=columns_to_drop)

# combined_df.to_csv("cocktail_dataset.csv")
# #Exporting a refined dataset to be used in mongoDB

# to_keep = ['recipe_id','rating','name','n_steps','ingredients','steps','n_ingredients']

# mongo_csv = combined_df[to_keep].drop_duplicates(subset=['name'])


# mongo_csv.to_csv("cocktail_data_mongo.csv", index=False)




## Data Exploration

We are exploring the cocktail counts

In [ ]:
# combined_df = pd.read_csv("cocktail_dataset.csv")

# cocktail_review_counts = combined_df.groupby('recipe_id')['rating'].count().reset_index()
# cocktail_review_counts.columns = ['recipe_id', 'review_count']
# # Sort the DataFrame by review_count in descending order
# # Sort the review counts DataFrame by review_count in descending order
# cocktail_review_counts_sorted = cocktail_review_counts.sort_values(by='review_count', ascending=False)

# # Merge the review counts DataFrame with the recipes DataFrame to get cocktail names
# table_data = cocktail_review_counts_sorted.merge(df_recipes[['recipe_id', 'name']], on='recipe_id', how='inner')

# # Display the table
# table = tabulate(table_data, headers=['Recipe ID','Review Count','Cocktail Name'], tablefmt='pretty', showindex=False)
# print(table)

# print(cocktail_review_counts_sorted['review_count'].mean())



## Data Preperation for Matrix

We Strip the regex characters from the ingredients column.

We also need to remove the duplicates since one cocktail can have many reviews, and since we combined the reviews and coctail dataframes, there will be duplicates.

The duplicates were useful for data exploration, but now exploration is done we need only unique values of the cocktails for our prediction matrix.

In [ ]:
# Only process if the first item in 'ingredients' column is a string
if isinstance(combined_df['ingredients'].iloc[0], str):
    # Remove [ and ] 
    combined_df['ingredients'] = combined_df['ingredients'].str.replace('[', '').str.replace(']', '')
    # Splitting the ingredients string by commas
    combined_df['ingredients'] = combined_df['ingredients'].str.split(',')


# Drop duplicates based on the 'name' column and assign the result back to combined_df
combined_df = combined_df.drop_duplicates(subset=['name'])

In [ ]:
#Checking if there are any duplicates before processing for the matrix

duplicate_names = combined_df[combined_df['name'].duplicated(keep=False)]
print(duplicate_names)


In [ ]:
# Explode the 'ingredients' column
exploded_df = combined_df.explode('ingredients')

# Find unique ingredients
unique_ingredients = exploded_df['ingredients'].unique()

ingredient_counts = exploded_df['ingredients'].value_counts()

exploded_df['ingredients'] = exploded_df['ingredients'].str.replace('','')

print(ingredient_counts)



# Cocktail Similarity
## Next We will prepare the features of the data
**Feature Extraction**
The ingredients list will be the primary feature for our content-based filtering.
**One-hot encoding**
This converts our categorical data into a numerical format that machine learning algorithms can understand and process.



In [ ]:
# Instantiate the binarizer
mlb = MultiLabelBinarizer()

# Filtering out non-iterable items
filtered_ingredients = [ingredients if isinstance(ingredients, (list, tuple)) else [] for ingredients in combined_df['ingredients']]

# Apply MultiLabelBinarizer on the filtered data
binary_matrix = mlb.fit_transform(filtered_ingredients)

# Convert the binary matrix into a DataFrame for better visualization and manipulation
df_binary = pd.DataFrame(binary_matrix, columns=mlb.classes_)


combined_df = pd.concat([combined_df, df_binary], axis=1)

combined_df = combined_df.reset_index(drop=True)


combined_df.head()

In [ ]:
similarity_matrix = cosine_similarity(df_binary)


In [ ]:
def get_similar_cocktails(input_value, N=5):
    """
    Fetch similar cocktails based on a given cocktail name or ID.
    
    Args:
    - input_value (str or int): Name or ID of the cocktail.
    - N (int): Number of similar cocktails to return. Default is 5.

    Returns:
    - list: Names of top N similar cocktails.
    """
    
    # Determine if input is name or ID
    if isinstance(input_value, str):
        if input_value not in combined_df['name'].values:
            raise ValueError(f"No cocktail named {input_value} found in the dataset.")
        cocktail_index = combined_df[combined_df['name'] == input_value].index[0]
    elif isinstance(input_value, int):  # Assuming ID is an integer
        if input_value not in combined_df['recipe_id'].values:
            raise ValueError(f"No cocktail with ID {input_value} found in the dataset.")
        cocktail_index = combined_df[combined_df['recipe_id'] == input_value].index[0]
    else:
        raise ValueError("Input value must be either a name (string) or an ID (integer).")
    
    # Fetch and enumerate similarity scores for the given cocktail
    similar_scores = list(enumerate(similarity_matrix[cocktail_index]))
    
    # Sort the scores
    sorted_similar_scores = sorted(similar_scores, key=lambda x: x[1], reverse=True)
    
    # Return the top N cocktail names excluding the input cocktail itself
    return [combined_df.iloc[i[0]]['name'] for i in sorted_similar_scores[1:N+1]]

get_similar_cocktails(98221)


## Web Scraping

The provided code demonstrates an automated process for extracting the first image from Google Image search results based on a list of cocktail-related queries.

---

### Libraries and Initialization:

**Selenium** is used for web automation. It opens a Chrome browser and interacts with web pages.  
**Requests** and **io** libraries facilitate the downloading of images from the internet.  
**PIL** from the Pillow library aids in image manipulation.

---

### Main Functions:

**1. `get_first_image_from_google`:**  
This function searches Google Images with a given query.  
It clicks on the first image thumbnail to view it in full.  
The direct URL of the image is then extracted and returned.

**2. `download_image`:**  
Given an image URL, this function fetches the image using the requests library.  
The image is then saved to the local disk using the Pillow library.

---

### Execution:

**Queries Creation:**  
The code prepares a list of queries named `queries`, where each query is generated by appending "food.com cocktail" to each 'name' from the `combined_df` DataFrame.

**Image Extraction:**  
The code demonstrates two potential methods:  
- Looping through the entire list of queries to download images for each one.
- Looping through a limited number (e.g., first 5) of queries from the list.

In this script, the second method is active. The code performs an image search for the first n cocktails from the list, fetches their first images, and saves them with a numerical filename (0.jpg, 1.jpg, etc.).

After all operations, the automated browser session (`wd`) is closed using `wd.quit()`.




In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
import io
from PIL import Image
import time


# See README for setup of chromedriver

PATH = r"C:\Users\samue\OneDrive\Documents\GitHub\software-innovation-studio\modelGeneration\chromedriver.exe"
wd = webdriver.Chrome(PATH)

def get_first_image_from_google(wd, query):
    # Format the URL with the given query
    url = f"https://www.google.com/search?q={query}&tbm=isch"
    wd.get(url)
    
    try:
        # Find the first image thumbnail and click it
        thumbnail = wd.find_element(By.CLASS_NAME, "Q4LuWd")
        thumbnail.click()
        time.sleep(1)  # Wait for the image to load

        # Extract the image URL
        image = wd.find_element(By.CLASS_NAME, "r48jcc")
        if image.get_attribute('src') and 'http' in image.get_attribute('src'):
            return image.get_attribute('src')
    except Exception as e:
        print(f"Error: {e}")

    return None

def download_image(download_path, url, file_name):
    try:
        image_content = requests.get(url).content
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file)
        file_path = download_path + file_name

        with open(file_path, "wb") as f:
            image.save(f, "JPEG")

        print("Success")
    except Exception as e:
        print('FAILED -', e)

queries = [f"{row['name']} food.com cocktail" for _, row in combined_df.iterrows()]

# This Loops through entire dataset of cocktails
# for i in range(len(queries)):
#     url = get_first_image_from_google(wd, queries[i])
#     if url:
#         download_image("", url, f"{i}.jpg")

#Just does n amount
n = 5
for i in range(n):
    url = get_first_image_from_google(wd, queries[i])
    if url:
        download_image("", url, f"{i}.jpg")


wd.quit()


WebDriverException: Message: 'chromedriver.exe' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home
